# Project 1 - Option B

### Monte Carlo simulations for modelling a particle physics experiment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [26]:
# generate a beam of particles with velocities selected from a normal distribution
# set their inityial positions equal to zero, working in 3D

def generate_beam(mean_v, std_v):
    velocity = np.array([ 0, 0, np.random.normal(mean_v, std_v)])
    position = np.zeros((1,3))
    return position, velocity


# test function
position, velocity = generate_beam(2000, 100)

print(velocity)
print(position)

[   0.            0.         1848.48837617]
[[0. 0. 0.]]


In [27]:
# create a function that generates decay times for particles based on an exponential distribution
# then calculates their decay positions based on their velocities and the dcay time

def decay_position(Lambda, position):
    d_time = np.random.exponential(1/Lambda) # CHECK function - is it the correct exponential?
    decay_position_z = d_time * velocity[2]
    position[0][2] = decay_position_z
    return position


# test function
decay = decay_position(0.5, position)

print(decay)


[[  0.           0.         716.05676402]]


In [28]:
# function to select the daughter particle direction based on it being isotropic

def daughter_initial_direction():
    theta = np.arccos(1 - 2 * np.random.rand())
    phi = 2 * np.pi * np.random.rand()
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    direction = np.array([x, y, z])
    direction = direction / np.linalg.norm(direction)
    return direction

# test function
direction = daughter_initial_direction()
print(direction)


[ 0.30628123  0.53705067 -0.78598243]


In [36]:
# function to track the daughter particle at each tracking station

def daughter_position_eqn(z, direction, decay):

    m_x = direction[0] / direction[2]
    m_y = direction[1] / direction[2]

    c_x = decay[0][0] - m_x * decay[0][2]
    c_y = decay[0][1] - m_y * decay[0][2]

    x = m_x * z + c_x
    y = m_y * z + c_y

    
    position = np.array([x, y, z])
    return position, m_x, m_y, c_x, c_y

# test function
position = daughter_position_eqn(40, direction, decay)
print(position)



(array([263.44545432, 461.94002044,  40.        ]), np.float64(-0.3896794889817873), np.float64(-0.683285849693451), np.float64(279.03263388371244), np.float64(489.27145442942475))


In [37]:
# function to bring all components together to run the simulation

def simulation(num_particles, mean_v, std_v, Lambda, detector_positions):
    detected_positions = {z: [] for z in detector_positions}

    for _ in range(num_particles):
        position, velocity = generate_beam(mean_v, std_v)
        decay = decay_position(Lambda, position)
        direction = daughter_initial_direction()

        for z in detector_positions:
            daughter_pos, m_x, m_y, c_x, c_y = daughter_position_eqn(z, direction, decay)
            detected_positions[z].append(daughter_pos)
    
        print(f'x = {m_x} * z + {c_x}')
        print(f'y = {m_y} * z + {c_y}')

    return detected_positions

In [38]:
#running the simulation with the given parameters

num_particles = 1
mean_v = 2000
std_v = 100
Lambda = 1/0.0025
detector_positions = [30, 35, 40, 45]


run = simulation(num_particles, mean_v, std_v, Lambda, detector_positions)
print(run)
    

x = -0.6397104928552131 * z + 0.19391766491803072
y = -0.11510636594786722 * z + 0.03489259274486018
{30: [array([-18.99739712,  -3.41829839,  30.        ])], 35: [array([-22.19594959,  -3.99383022,  35.        ])], 40: [array([-25.39450205,  -4.56936205,  40.        ])], 45: [array([-28.59305451,  -5.14489387,  45.        ])]}
